# Cold Face Test – Heart Rate Plots

In [ ]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps, colors_all, register_fausans_font
import biopsykit as bp
from biopsykit.protocols import MIST
from biopsykit.utils.dataframe_handling import multi_xs

from cft_analysis.datasets import CftDatasetProcessed

from IPython.display import Markdown

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

register_fausans_font()
palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

## Data Import

In [ ]:
# get path to analysis results
data_path = Path("../../2022_scientific_reports/data")
base_path = Path("..")

In [ ]:
results_path = base_path.joinpath("results")
plot_path = results_path.joinpath("plots")
bp.utils.file_handling.mkdirs([results_path, plot_path])

In [ ]:
hue_order = ["Control", "CFT"]

subphase_order = ["BL", "RP/CFI", "AT", "FB"]

In [ ]:
dataset = CftDatasetProcessed(data_path, exclude_subjects=True)
dataset

In [ ]:
hr_ensemble = dataset.heart_rate_ensemble
hr_mean = dataset.heart_rate
hr_mean = hr_mean.xs("HR_Norm", level="type")
hr_mean.head()

In [ ]:
mist = MIST.from_file(data_path.joinpath("mist_cft.json"))
mist

In [ ]:
mist.add_hr_ensemble("hr_ensemble", {key: val.xs(key) for key, val in hr_ensemble.groupby("phase")})

hr_mean = hr_mean.rename({"RP_CFI": "RP/CFI"}, level="subphase").reindex(subphase_order, level="subphase")
mist.add_hr_results("hr_mean", hr_mean)
mist.add_hr_results("hr_mean_control", hr_mean.xs("Control", drop_level=False))

In [ ]:
mist

## HR Mean Plot

In [ ]:
fig, ax = plt.subplots()

# HR Mean Plot
mist.hr_mean_plot("hr_mean_control", is_relative=True, ax=ax, order=["Control"])
ax.set_ylim([-15, 32.5])
fig.tight_layout(pad=0.1)

fig.savefig(plot_path.joinpath("img_hr_mean_course_control.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots()

# HR Mean Plot
mist.hr_mean_plot("hr_mean", is_relative=True, ax=ax, order=hue_order)
ax.set_ylim([-15, 32.5])
fig.tight_layout(pad=0.1)

fig.savefig(plot_path.joinpath("img_hr_mean_course.pdf"), transparent=True)